In [ ]:
import pandas as pd

In [ ]:
%run ../tasking_manager_stats/data_management

In [ ]:
df = pd.read_csv(os.path.join(get_data_dir(), 'merged_stats.csv'))
df['date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-' + df['Day'].astype(str)
df['date'] = pd.to_datetime(df['date'], yearfirst=True)
df.head()

In [ ]:
df2 = df[(df['Hour'] > 17) & (df['Hour'] < 22)]
df2.head()

In [ ]:
mapathons = pd.read_csv(os.path.join('..', 'data', 'Mapathons_2020_02_08.csv'))
mapathons['Date'] = pd.to_datetime(mapathons['Date'], dayfirst=True)
mapathons.head()

In [ ]:
mapathons2 = pd.DataFrame()
for _, row in mapathons.iterrows():
    # Extract projects of the mapathon
    tasks = row['Tasks']
    if pd.isnull(tasks):
        continue
    projects = set()
    for s in tasks.split('/'):
        try:
            projects.add(int(s))
        except:
            pass
    for s in tasks.split(', '):
        try:
            projects.add(int(s))
        except:
            pass
    # Create new mapathon line for each project
    for project in projects:
        mapathons2 = pd.concat([mapathons2, pd.DataFrame(data=[(row['Date'], row['City'], project)], columns=['date', 'City', 'Project'])], axis=0, ignore_index=True)

In [ ]:
mapathons2.head()

In [ ]:
len(df2)

In [ ]:
len(mapathons2)

In [ ]:
df3 = pd.merge(df2.loc[df2['Type'] == 'MAPPING'], mapathons2, on=['date', 'Project'])

In [ ]:
len(df3)

In [ ]:
df3.head()

In [ ]:
df4 = df3[['date', 'Author', 'City', 'Project']].drop_duplicates()
df4.head()

In [ ]:
len(df4)

In [ ]:
tmp = df4[['Author', 'City']].drop_duplicates()
tmp.loc[tmp['Author'] == 'NicolasGrosjean']

In [ ]:
df4.loc[df4['Author'] == 'NicolasGrosjean']

In [ ]:
df5 = df4[['Author', 'date']].drop_duplicates().groupby('Author').count().date.reset_index()

In [ ]:
df5.loc[df5['Author'] == 'NicolasGrosjean']

In [ ]:
def compute_mapathon_number(mapathon_file, stats_file, max_date=None):
    mapathons = pd.read_csv(mapathon_file)
    mapathons['Date'] = pd.to_datetime(mapathons['Date'], dayfirst=True)

    # Extract projects of the mapathon
    mapathons2 = pd.DataFrame()
    for _, row in mapathons.iterrows():
        tasks = row['Tasks']
        if pd.isnull(tasks):
            continue
        projects = set()
        for s in tasks.split('/'):
            try:
                projects.add(int(s))
            except:
                pass
        for s in tasks.split(', '):
            try:
                projects.add(int(s))
            except:
                pass  
        # Create new mapathon line for each project
        for project in projects:
            mapathons2 = pd.concat([mapathons2, pd.DataFrame(data=[(row['Date'], row['City'], project)], columns=['date', 'City', 'Project'])], axis=0, ignore_index=True)
        
    # Compute number of mapathons by contributor
    df = pd.read_csv(stats_file)
    df['date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-' + df['Day'].astype(str)
    df['date'] = pd.to_datetime(df['date'], yearfirst=True)
    if max_date is not None:
        df = df[df['date'] <= max_date]
    df2 = df[(df['Hour'] > 17) & (df['Hour'] < 22)]
    df3 = pd.merge(df2.loc[df2['Type'] == 'MAPPING'], mapathons2, on=['date', 'Project'])
    df4 = df3[['date', 'Author', 'City', 'Project']].drop_duplicates()
    return df4[['Author', 'date']].drop_duplicates().groupby('Author').count().date.reset_index()

In [ ]:
df6 = compute_mapathon_number(os.path.join('..', 'data', 'Mapathons_2020_02_08.csv'),
                              os.path.join(get_data_dir(), 'merged_stats.csv'))
df6.loc[df6['Author'] == 'NicolasGrosjean']

In [ ]:
max_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
df6 = compute_mapathon_number(os.path.join('..', 'data', 'Mapathons_2020_02_08.csv'),
                              os.path.join(get_data_dir(), 'merged_stats.csv'), max_date)
df6.loc[df6['Author'] == 'NicolasGrosjean']

# Validation

- WARNING : Divide validation time by simultanous validation tasks
- Own mapping and validation time
- Validation time on own mapping tasks

In [ ]:
df_project = pd.read_csv(os.path.join(get_data_dir(), 'stats', '6055.csv'))
df_project = df_project[df_project['Type'] == 'VALIDATION']
df_project.head()

In [ ]:
key = ['Year', 'Month', 'Day', 'Rel. Day', 'Hour', 'Minute', 'Second', 'Duration', 'Author', 'Type']
df_project2 = df_project.groupby(key).count().Task
df_project2 = df_project2.reset_index()
df_project2.tail()

In [ ]:
df_project3 = pd.merge(df_project, df_project2, on=key)
df_project3.tail()

In [ ]:
df_project3['Duration'] /= df_project3['Task_y']

In [ ]:
df_project3['Duration'].sum()

In [ ]:
df_project['Duration'].sum() - 18*336

In [ ]:
def compute_validation_time_by_task(stats_dir, csv_file, max_date=None):
    df_project = pd.read_csv(os.path.join(stats_dir, csv_file))
    df_project = df_project[df_project['Type'] == 'VALIDATION']
    if len(df_project) == 0:
        return pd.DataFrame()
    if max_date is not None:
        df_project['date'] = df_project['Year'].astype(str) + '-' + df_project['Month'].astype(str) + '-' + df_project['Day'].astype(str)
        df_project['date'] = pd.to_datetime(df_project['date'], yearfirst=True)
        df_project = df_project[df_project['date'] <= max_date]
        if len(df_project) == 0:
            return pd.DataFrame()
    key = ['Year', 'Month', 'Day', 'Rel. Day', 'Hour', 'Minute', 'Second', 'Duration', 'Author', 'Type']
    df_project2 = df_project.groupby(key).count().Task
    df_project2 = df_project2.reset_index()
    df_project3 = pd.merge(df_project, df_project2, on=key)
    df_project3['Duration'] /= df_project3['Task_y']
    df_project3 = df_project3[['Project', 'Task_x', 'Duration']]
    df_project3.columns = ['Project', 'Task', 'Duration']
    df_project3 = df_project3.groupby(['Project', 'Task']).sum().reset_index()
    return df_project3

In [ ]:
compute_validation_time_by_task(os.path.join(get_data_dir(), 'stats'), '6055.csv')['Duration'].sum()

In [ ]:
def compute_validation_time_by_task_all_projects(stats_dir, max_date=None):
    df = pd.DataFrame()
    for file in tqdm(os.listdir(stats_dir)):
        if file.endswith('.csv'):
            df = pd.concat([df, compute_validation_time_by_task(stats_dir, file, max_date)], axis=0, ignore_index=True)
    return df

In [ ]:
validation_df = compute_validation_time_by_task_all_projects(os.path.join(get_data_dir(), 'stats'))
validation_df.columns = ['Project', 'Task', 'ValidationDuration']
validation_df.tail()

In [ ]:
df_one_author = pd.read_csv(os.path.join(get_data_dir(), 'merged_stats_one_author_by_task_type.csv'))
df_one_author.head()

In [ ]:
df_mapping = pd.merge(validation_df, df_one_author[df_one_author['Type'] == 'MAPPING'], on=['Project', 'Task'])
df_mapping.tail()

In [ ]:
len(df_mapping)

In [ ]:
len(df_mapping[['Project', 'Task']].drop_duplicates())

In [ ]:
df_mapping_valid = df_mapping.groupby('Author').sum().ValidationDuration.reset_index()
df_mapping_valid.sort_values('ValidationDuration', ascending=False).head()

In [ ]:
df_pure_mapping = df[df['Type'] == 'MAPPING'].groupby('Author').sum().Duration.reset_index()
df_pure_mapping.columns = ['Author', 'OwnMappingDuration']
df_pure_mapping.sort_values('OwnMappingDuration', ascending=False).head()

In [ ]:
df_pure_validation = df[df['Type'] == 'VALIDATION'].groupby('Author').sum().Duration.reset_index()
df_pure_validation.columns = ['Author', 'OwnValidationDuration']
df_pure_validation.sort_values('OwnValidationDuration', ascending=False).head()

In [ ]:
def compute_advanced_stats(stats_dir, df_one_author, df, max_date=None):
    validation_df = compute_validation_time_by_task_all_projects(stats_dir, max_date)
    validation_df.columns = ['Project', 'Task', 'ValidationDuration']
    df_mapping = pd.merge(validation_df, df_one_author[df_one_author['Type'] == 'MAPPING'], on=['Project', 'Task'])
    df_mapping_valid = df_mapping.groupby('Author').sum().ValidationDuration.reset_index()
    df_pure_mapping = df[df['Type'] == 'MAPPING'].groupby('Author').sum().Duration.reset_index()
    df_pure_mapping.columns = ['Author', 'OwnMappingDuration']
    res = pd.merge(df_pure_mapping, df_mapping_valid, on='Author', how='left')
    df_pure_validation = df[df['Type'] == 'VALIDATION'].groupby('Author').sum().Duration.reset_index()
    df_pure_validation.columns = ['Author', 'OwnValidationDuration']
    res = pd.merge(res, df_pure_validation, on='Author', how='left')
    res['ValidationOnOwnMappingDuration'] = res['ValidationDuration'] / res['OwnMappingDuration']
    return res

In [ ]:
df_adv = compute_advanced_stats(os.path.join(get_data_dir(), 'stats'), df_one_author, df)
df_adv.sort_values('ValidationOnOwnMappingDuration').head(10)

In [ ]:
df_adv[(~pd.isnull(df_adv['ValidationOnOwnMappingDuration'])) & (df_adv['OwnMappingDuration'] > 3600)].sort_values('ValidationOnOwnMappingDuration').tail(10)

In [ ]:
df_adv[df_adv['ValidationDuration'] > 3600].sort_values('ValidationOnOwnMappingDuration').head(20)

In [ ]:
max_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
df_adv = compute_advanced_stats(os.path.join(get_data_dir(), 'stats'), df_one_author, df, max_date)
df_adv[df_adv['ValidationDuration'] > 3600].sort_values('ValidationOnOwnMappingDuration').head(20)